<a href="https://colab.research.google.com/github/wlg100/numseqcont_circuit_expms/blob/main/notebook_templates/minimal_circuit_template.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" align="left"/></a>&nbsp;or in a local notebook.

# Setup
(No need to change anything)

In [1]:
# Janky code to do different setup when run in a Colab notebook vs VSCode
DEBUG_MODE = False
try:
    import google.colab
    IN_COLAB = True
    print("Running as a Colab notebook")
    %pip install git+https://github.com/neelnanda-io/TransformerLens.git
    # Install another version of node that makes PySvelte work way faster
    # !curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs
    # %pip install git+https://github.com/neelnanda-io/PySvelte.git
except:
    IN_COLAB = False
    print("Running as a Jupyter notebook - intended for development only!")
    from IPython import get_ipython

    ipython = get_ipython()
    # Code to automatically update the HookedTransformer code as its edited without restarting the kernel
    ipython.magic("load_ext autoreload")
    ipython.magic("autoreload 2")

Running as a Colab notebook
  Cloning https://github.com/neelnanda-io/TransformerLens.git to /tmp/pip-req-build-qe1ipykl
  Running command git clone --filter=blob:none --quiet https://github.com/neelnanda-io/TransformerLens.git /tmp/pip-req-build-qe1ipykl
  Resolved https://github.com/neelnanda-io/TransformerLens.git to commit fa287750606075574df2c538058e67d648e2f952
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.7/739.7 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 49.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 89.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 2.5 MB/s eta 0:00:00
    

In [2]:
# Plotly needs a different renderer for VSCode/Notebooks vs Colab argh
import plotly.io as pio

if IN_COLAB or not DEBUG_MODE:
    # Thanks to annoying rendering issues, Plotly graphics will either show up in colab OR Vscode depending on the renderer - this is bad for developing demos! Thus creating a debug mode.
    pio.renderers.default = "colab"
else:
    pio.renderers.default = "png"

In [3]:
# Import stuff
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import einops
from fancy_einsum import einsum
import tqdm.notebook as tqdm
import random
from pathlib import Path
import plotly.express as px
from torch.utils.data import DataLoader

from jaxtyping import Float, Int
from typing import List, Union, Optional
from functools import partial
import copy

import itertools
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import dataclasses
import datasets
from IPython.display import HTML

In [4]:
# import pysvelte

import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache

We turn automatic differentiation off, to save GPU memory, as this notebook focuses on model inference not model training.

In [5]:
torch.set_grad_enabled(False)

Plotting helper functions:

In [6]:
def imshow(tensor, renderer=None, **kwargs):
    px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0, color_continuous_scale="RdBu", **kwargs).show(renderer)

def line(tensor, renderer=None, **kwargs):
    px.line(y=utils.to_numpy(tensor), **kwargs).show(renderer)

def scatter(x, y, xaxis="", yaxis="", caxis="", renderer=None, **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(y=y, x=x, labels={"x":xaxis, "y":yaxis, "color":caxis}, **kwargs).show(renderer)

## Load Model

Decide which model to use (eg. gpt2-small vs -medium)

In [7]:
model = HookedTransformer.from_pretrained(
    "gpt2-small",
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    refactor_factored_attn_matrices=True,
)

Loaded pretrained model gpt2-small into HookedTransformer


## Import functions from repo

In [8]:
!git clone https://github.com/callummcdougall/ARENA_2.0.git

Cloning into 'ARENA_2.0'...
remote: Enumerating objects: 9106, done.
remote: Counting objects: 100% (1820/1820), done.
remote: Compressing objects: 100% (289/289), done.
remote: Total 9106 (delta 1614), reused 1608 (delta 1528), pack-reused 7286
Receiving objects: 100% (9106/9106), 155.60 MiB | 37.94 MiB/s, done.
Resolving deltas: 100% (5507/5507), done.


In [9]:
cd ARENA_2.0/chapter1_transformers/exercises/part3_indirect_object_identification

/content/ARENA_2.0/chapter1_transformers/exercises/part3_indirect_object_identification


In [10]:
import ioi_circuit_extraction as ioi_circuit_extraction

# test prompts

In [11]:
example_prompt = "100 99 98 97"
example_answer = " 96"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '100', ' 99', ' 98', ' 97']
Tokenized answer: [' 96']


Performance on answer token:
Rank: 0        Logit: 20.37 Prob: 54.17% Token: | 96|

Top 0th token. Logit: 20.37 Prob: 54.17% Token: | 96|
Top 1th token. Logit: 18.67 Prob:  9.82% Token: | 97|
Top 2th token. Logit: 18.34 Prob:  7.09% Token: | 98|
Top 3th token. Logit: 18.20 Prob:  6.13% Token: | 95|
Top 4th token. Logit: 17.48 Prob:  2.98% Token: |
|
Top 5th token. Logit: 17.35 Prob:  2.62% Token: | 94|
Top 6th token. Logit: 17.34 Prob:  2.61% Token: | 92|
Top 7th token. Logit: 16.81 Prob:  1.53% Token: |.|
Top 8th token. Logit: 16.74 Prob:  1.44% Token: | 100|
Top 9th token. Logit: 16.33 Prob:  0.95% Token: | 99|


Ranks of the answer tokens: [(' 96', 0)]

In [12]:
example_prompt = "58 57 56 55"
example_answer = " 54"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '58', ' 57', ' 56', ' 55']
Tokenized answer: [' 54']


Performance on answer token:
Rank: 0        Logit: 18.82 Prob: 26.37% Token: | 54|

Top 0th token. Logit: 18.82 Prob: 26.37% Token: | 54|
Top 1th token. Logit: 17.63 Prob:  8.06% Token: | 56|
Top 2th token. Logit: 17.37 Prob:  6.18% Token: | 55|
Top 3th token. Logit: 17.30 Prob:  5.77% Token: | 44|
Top 4th token. Logit: 17.16 Prob:  5.04% Token: | 52|
Top 5th token. Logit: 16.99 Prob:  4.23% Token: | 53|
Top 6th token. Logit: 16.88 Prob:  3.80% Token: | 46|
Top 7th token. Logit: 16.52 Prob:  2.64% Token: |
|
Top 8th token. Logit: 16.51 Prob:  2.63% Token: | 51|
Top 9th token. Logit: 16.45 Prob:  2.48% Token: | 50|


Ranks of the answer tokens: [(' 54', 0)]

In [28]:
example_prompt = "104 103 102 101"
example_answer = " 100"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '104', ' 103', ' 102', ' 101']
Tokenized answer: [' 100']


Performance on answer token:
Rank: 0        Logit: 18.19 Prob: 23.47% Token: | 100|

Top 0th token. Logit: 18.19 Prob: 23.47% Token: | 100|
Top 1th token. Logit: 17.87 Prob: 17.05% Token: | 101|
Top 2th token. Logit: 17.14 Prob:  8.23% Token: | 102|
Top 3th token. Logit: 16.69 Prob:  5.24% Token: | 103|
Top 4th token. Logit: 16.57 Prob:  4.64% Token: |
|
Top 5th token. Logit: 16.54 Prob:  4.52% Token: | 99|
Top 6th token. Logit: 16.42 Prob:  4.01% Token: | 90|
Top 7th token. Logit: 16.30 Prob:  3.55% Token: | 98|
Top 8th token. Logit: 15.79 Prob:  2.14% Token: | 97|
Top 9th token. Logit: 15.75 Prob:  2.05% Token: | 0|


Ranks of the answer tokens: [(' 100', 0)]

In [29]:
example_prompt = "105 104 103 102 101"
example_answer = " 100"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '105', ' 104', ' 103', ' 102', ' 101']
Tokenized answer: [' 100']


Performance on answer token:
Rank: 0        Logit: 19.31 Prob: 30.24% Token: | 100|

Top 0th token. Logit: 19.31 Prob: 30.24% Token: | 100|
Top 1th token. Logit: 18.95 Prob: 21.14% Token: | 101|
Top 2th token. Logit: 17.99 Prob:  8.10% Token: | 90|
Top 3th token. Logit: 17.71 Prob:  6.11% Token: | 99|
Top 4th token. Logit: 17.51 Prob:  5.05% Token: | 102|
Top 5th token. Logit: 17.35 Prob:  4.27% Token: |
|
Top 6th token. Logit: 16.84 Prob:  2.58% Token: | 98|
Top 7th token. Logit: 16.43 Prob:  1.70% Token: | 103|
Top 8th token. Logit: 16.36 Prob:  1.60% Token: | 97|
Top 9th token. Logit: 16.32 Prob:  1.53% Token: | 91|


Ranks of the answer tokens: [(' 100', 0)]

In [30]:
example_prompt = "110 109 108 107 106 105 104 103"
example_answer = " 102"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '110', ' 109', ' 108', ' 107', ' 106', ' 105', ' 104', ' 103']
Tokenized answer: [' 102']


Performance on answer token:
Rank: 0        Logit: 23.03 Prob: 83.55% Token: | 102|

Top 0th token. Logit: 23.03 Prob: 83.55% Token: | 102|
Top 1th token. Logit: 20.43 Prob:  6.24% Token: | 103|
Top 2th token. Logit: 19.71 Prob:  3.04% Token: | 104|
Top 3th token. Logit: 18.68 Prob:  1.08% Token: | 101|
Top 4th token. Logit: 18.67 Prob:  1.08% Token: |
|
Top 5th token. Logit: 18.49 Prob:  0.89% Token: | 92|
Top 6th token. Logit: 18.16 Prob:  0.64% Token: | 100|
Top 7th token. Logit: 17.38 Prob:  0.29% Token: | 98|
Top 8th token. Logit: 16.94 Prob:  0.19% Token: |

|
Top 9th token. Logit: 16.84 Prob:  0.17% Token: | Clean|


Ranks of the answer tokens: [(' 102', 0)]

In [32]:
example_prompt = "110 109 108 107 106 105 104 103 102 101"
example_answer = " 100"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '110', ' 109', ' 108', ' 107', ' 106', ' 105', ' 104', ' 103', ' 102', ' 101']
Tokenized answer: [' 100']


Performance on answer token:
Rank: 0        Logit: 20.99 Prob: 37.51% Token: | 100|

Top 0th token. Logit: 20.99 Prob: 37.51% Token: | 100|
Top 1th token. Logit: 20.53 Prob: 23.72% Token: | 101|
Top 2th token. Logit: 19.97 Prob: 13.60% Token: | 90|
Top 3th token. Logit: 19.03 Prob:  5.29% Token: |
|
Top 4th token. Logit: 18.63 Prob:  3.56% Token: | 99|
Top 5th token. Logit: 17.86 Prob:  1.65% Token: | 98|
Top 6th token. Logit: 17.61 Prob:  1.28% Token: |<|endoftext|>|
Top 7th token. Logit: 17.60 Prob:  1.26% Token: | 102|
Top 8th token. Logit: 17.14 Prob:  0.80% Token: | 0|
Top 9th token. Logit: 17.03 Prob:  0.72% Token: | 89|


Ranks of the answer tokens: [(' 100', 0)]

In [33]:
example_prompt = "110 109 108 107 106 105 104 103 102 101 100"
example_answer = " 99"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '110', ' 109', ' 108', ' 107', ' 106', ' 105', ' 104', ' 103', ' 102', ' 101', ' 100']
Tokenized answer: [' 99']


Performance on answer token:
Rank: 0        Logit: 19.51 Prob: 55.29% Token: | 99|

Top 0th token. Logit: 19.51 Prob: 55.29% Token: | 99|
Top 1th token. Logit: 17.54 Prob:  7.74% Token: | 89|
Top 2th token. Logit: 17.47 Prob:  7.23% Token: |
|
Top 3th token. Logit: 17.06 Prob:  4.82% Token: | 100|
Top 4th token. Logit: 16.38 Prob:  2.42% Token: | 97|
Top 5th token. Logit: 15.98 Prob:  1.63% Token: |<|endoftext|>|
Top 6th token. Logit: 15.97 Prob:  1.62% Token: | No|
Top 7th token. Logit: 15.69 Prob:  1.22% Token: | 101|
Top 8th token. Logit: 15.59 Prob:  1.11% Token: | Top|
Top 9th token. Logit: 15.43 Prob:  0.93% Token: | 0|


Ranks of the answer tokens: [(' 99', 0)]

In [34]:
example_prompt = "107 106 105 104 103 102 101 100"
example_answer = " 99"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '107', ' 106', ' 105', ' 104', ' 103', ' 102', ' 101', ' 100']
Tokenized answer: [' 99']


Performance on answer token:
Rank: 0        Logit: 18.19 Prob: 37.24% Token: | 99|

Top 0th token. Logit: 18.19 Prob: 37.24% Token: | 99|
Top 1th token. Logit: 17.29 Prob: 15.08% Token: | 100|
Top 2th token. Logit: 16.55 Prob:  7.25% Token: | 89|
Top 3th token. Logit: 16.51 Prob:  6.97% Token: |
|
Top 4th token. Logit: 16.10 Prob:  4.62% Token: | 97|
Top 5th token. Logit: 15.36 Prob:  2.20% Token: | 101|
Top 6th token. Logit: 15.00 Prob:  1.54% Token: | 98|
Top 7th token. Logit: 14.98 Prob:  1.50% Token: | No|
Top 8th token. Logit: 14.84 Prob:  1.31% Token: | 0|
Top 9th token. Logit: 14.32 Prob:  0.78% Token: | 91|


Ranks of the answer tokens: [(' 99', 0)]

In [35]:
example_prompt = "107 106 105 104 103 102 101 100 99"
example_answer = " 98"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '107', ' 106', ' 105', ' 104', ' 103', ' 102', ' 101', ' 100', ' 99']
Tokenized answer: [' 98']


Performance on answer token:
Rank: 0        Logit: 21.75 Prob: 58.19% Token: | 98|

Top 0th token. Logit: 21.75 Prob: 58.19% Token: | 98|
Top 1th token. Logit: 20.03 Prob: 10.42% Token: | 100|
Top 2th token. Logit: 19.41 Prob:  5.60% Token: | 96|
Top 3th token. Logit: 19.40 Prob:  5.56% Token: | 88|
Top 4th token. Logit: 19.10 Prob:  4.09% Token: | 99|
Top 5th token. Logit: 18.92 Prob:  3.45% Token: | 97|
Top 6th token. Logit: 17.81 Prob:  1.13% Token: |.|
Top 7th token. Logit: 17.78 Prob:  1.10% Token: | 89|
Top 8th token. Logit: 17.78 Prob:  1.10% Token: | 92|
Top 9th token. Logit: 17.69 Prob:  1.00% Token: | 80|


Ranks of the answer tokens: [(' 98', 0)]

In [36]:
example_prompt = "99 98 97 96 95"
example_answer = " 94"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '99', ' 98', ' 97', ' 96', ' 95']
Tokenized answer: [' 94']


Performance on answer token:
Rank: 0        Logit: 22.24 Prob: 54.84% Token: | 94|

Top 0th token. Logit: 22.24 Prob: 54.84% Token: | 94|
Top 1th token. Logit: 20.41 Prob:  8.81% Token: | 95|
Top 2th token. Logit: 20.33 Prob:  8.09% Token: | 92|
Top 3th token. Logit: 20.21 Prob:  7.17% Token: | 96|
Top 4th token. Logit: 20.17 Prob:  6.94% Token: | 93|
Top 5th token. Logit: 18.78 Prob:  1.73% Token: | 91|
Top 6th token. Logit: 18.49 Prob:  1.29% Token: |
|
Top 7th token. Logit: 18.41 Prob:  1.19% Token: | 90|
Top 8th token. Logit: 18.26 Prob:  1.02% Token: | 89|
Top 9th token. Logit: 18.15 Prob:  0.92% Token: | 84|


Ranks of the answer tokens: [(' 94', 0)]

In [37]:
example_prompt = "99 98 97 96 95 94 93 92"
example_answer = " 91"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '99', ' 98', ' 97', ' 96', ' 95', ' 94', ' 93', ' 92']
Tokenized answer: [' 91']


Performance on answer token:
Rank: 0        Logit: 22.33 Prob: 80.08% Token: | 91|

Top 0th token. Logit: 22.33 Prob: 80.08% Token: | 91|
Top 1th token. Logit: 19.94 Prob:  7.30% Token: | 93|
Top 2th token. Logit: 19.92 Prob:  7.19% Token: | 92|
Top 3th token. Logit: 17.81 Prob:  0.87% Token: | 90|
Top 4th token. Logit: 17.76 Prob:  0.82% Token: | 89|
Top 5th token. Logit: 17.25 Prob:  0.50% Token: |
|
Top 6th token. Logit: 16.71 Prob:  0.29% Token: | 87|
Top 7th token. Logit: 16.01 Prob:  0.14% Token: | 71|
Top 8th token. Logit: 15.88 Prob:  0.13% Token: | Any|
Top 9th token. Logit: 15.66 Prob:  0.10% Token: | 1991|


Ranks of the answer tokens: [(' 91', 0)]

In [31]:
example_prompt = "55 56 57 58"
example_answer = " 59"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '55', ' 56', ' 57', ' 58']
Tokenized answer: [' 59']


Performance on answer token:
Rank: 0        Logit: 18.23 Prob: 90.82% Token: | 59|

Top 0th token. Logit: 18.23 Prob: 90.82% Token: | 59|
Top 1th token. Logit: 14.05 Prob:  1.38% Token: | 69|
Top 2th token. Logit: 13.25 Prob:  0.62% Token: |
|
Top 3th token. Logit: 13.05 Prob:  0.51% Token: | 58|
Top 4th token. Logit: 12.74 Prob:  0.37% Token: | 61|
Top 5th token. Logit: 12.68 Prob:  0.35% Token: | 60|
Top 6th token. Logit: 12.36 Prob:  0.26% Token: | 57|
Top 7th token. Logit: 12.35 Prob:  0.25% Token: | 63|
Top 8th token. Logit: 11.97 Prob:  0.17% Token: | 49|
Top 9th token. Logit: 11.92 Prob:  0.16% Token: | 19|


Ranks of the answer tokens: [(' 59', 0)]

# Generate dataset with multiple prompts

In [14]:
class Dataset:
    def __init__(self, prompts, pos_dict, tokenizer, S1_is_first=False):
        self.prompts = prompts
        self.tokenizer = tokenizer
        self.N = len(prompts)
        self.max_len = max(
            [
                len(self.tokenizer(prompt["text"]).input_ids)
                for prompt in self.prompts
            ]
        )
        # all_ids = [prompt["TEMPLATE_IDX"] for prompt in self.ioi_prompts]
        all_ids = [0 for prompt in self.prompts] # only 1 template
        all_ids_ar = np.array(all_ids)
        self.groups = []
        for id in list(set(all_ids)):
            self.groups.append(np.where(all_ids_ar == id)[0])

        texts = [ prompt["text"] for prompt in self.prompts ]
        self.toks = torch.Tensor(self.tokenizer(texts, padding=True).input_ids).type(
            torch.int
        )
        self.io_tokenIDs = [
            self.tokenizer.encode(" " + prompt["corr"])[0] for prompt in self.prompts
        ]
        self.s_tokenIDs = [
            self.tokenizer.encode(" " + prompt["incorr"])[0] for prompt in self.prompts
        ]

        # word_idx: for every prompt, find the token index of each target token and "end"
        # word_idx is a tensor with an element for each prompt. The element is the targ token's ind at that prompt
        self.word_idx = {}
        for targ in [key for key in self.prompts[0].keys() if (key != 'text' and key != 'corr' and key != 'incorr')]:
            targ_lst = []
            for prompt in self.prompts:
                input_text = prompt["text"]
                tokens = model.tokenizer.tokenize(input_text)
                # if S1_is_first and targ == "S1":  # only use this if first token doesn't have space Ġ in front
                #     target_token = prompt[targ]
                # else:
                #     target_token = "Ġ" + prompt[targ]
                # target_index = tokens.index(target_token)
                target_index = pos_dict[targ]
                targ_lst.append(target_index)
            self.word_idx[targ] = torch.tensor(targ_lst)

        targ_lst = []
        for prompt in self.prompts:
            input_text = prompt["text"]
            tokens = self.tokenizer.tokenize(input_text)
            end_token_index = len(tokens) - 1
            targ_lst.append(end_token_index)
        self.word_idx["end"] = torch.tensor(targ_lst)

    def __len__(self):
        return self.N

NOTE: if the predicted token is not at high logit, then mean ablation can potentially lead to higher scores. Make sure the predicted token is near high 90s logit.

With 4 elems, gpt-2 small usually only gets to 50% for the correct token for decr, unlike incr, for 4 member seq. Often takes seqs of 10 elems to get to mid-90s (which is 4 seq of incr). So use 10 elems. See:

https://colab.research.google.com/drive/1ahWI9e0NMeAjdFNnj2vEj4d4aYIGsoNP#scrollTo=Nkbv00d0Wn2z&line=1&uniqifier=1

In [15]:
for i in range(21, 17, -1):
    print(i)

21
20
19
18


In [39]:
pos_dict = {
    'S1': 0,
    'S2': 1,
    'S3': 2,
    'S4': 3,
    'S5': 4,
    'S6': 5,
    'S7': 6,
    'S8': 7,
}

In [42]:
def generate_prompts_list(x ,y):
    prompts_list = []
    for i in range(x, y, -1):
        prompt_dict = {
            'S1': str(i),
            'S2': str(i-1),
            'S3': str(i-2),
            'S4': str(i-3),
            'S5': str(i-4),
            'S6': str(i-5),
            'S7': str(i-6),
            'S8': str(i-7),
            'corr': str(i-8),
            'incorr': str(i-7),
            'text': f"{i} {i-1} {i-2} {i-3} {i-4} {i-5} {i-6} {i-7}"
        }
        prompts_list.append(prompt_dict)
    return prompts_list

prompts_list = generate_prompts_list(100, 8)
dataset = Dataset(prompts_list, pos_dict, model.tokenizer, S1_is_first=True)
len(prompts_list)

92

In [43]:
prompts_list[-1]

{'S1': '9',
 'S2': '8',
 'S3': '7',
 'S4': '6',
 'S5': '5',
 'S6': '4',
 'S7': '3',
 'S8': '2',
 'corr': '1',
 'incorr': '2',
 'text': '9 8 7 6 5 4 3 2'}

In [58]:
import random

def generate_prompts_list_corr(x ,y):
    prompts_list = []
    for i in range(x, y, -1):
        r1 = random.randint(1, 100)
        r2 = random.randint(1, 100)
        r3 = random.randint(1, 100)
        r4 = random.randint(1, 100)
        r4 = random.randint(1, 100)
        r5 = random.randint(1, 100)
        r6 = random.randint(1, 100)
        while True:
            r7 = random.randint(1, 100)
            r8 = random.randint(1, 100)
            if r7 -1 != r8:
                break
        prompt_dict = {
            'S1': str(r1),
            'S2': str(r2),
            'S3': str(r3),
            'S4': str(r4),
            'S5': str(r5),
            'S6': str(r6),
            'S7': str(r7),
            'S8': str(r8),
            'corr': str(i-8),
            'incorr': str(i-7),
            'text': f"{r1} {r2} {r3} {r4} {r5} {r6} {r7} {r8}"
        }
        prompts_list.append(prompt_dict)
    return prompts_list

prompts_list_2 = generate_prompts_list_corr(100, 8)
dataset_2 = Dataset(prompts_list_2, pos_dict, model.tokenizer, S1_is_first=True)
prompts_list_2

[{'S1': '16',
  'S2': '68',
  'S3': '38',
  'S4': '78',
  'S5': '74',
  'S6': '69',
  'S7': '4',
  'S8': '58',
  'corr': '92',
  'incorr': '93',
  'text': '16 68 38 78 74 69 4 58'},
 {'S1': '20',
  'S2': '99',
  'S3': '9',
  'S4': '61',
  'S5': '92',
  'S6': '91',
  'S7': '83',
  'S8': '18',
  'corr': '91',
  'incorr': '92',
  'text': '20 99 9 61 92 91 83 18'},
 {'S1': '7',
  'S2': '31',
  'S3': '25',
  'S4': '96',
  'S5': '41',
  'S6': '2',
  'S7': '78',
  'S8': '28',
  'corr': '90',
  'incorr': '91',
  'text': '7 31 25 96 41 2 78 28'},
 {'S1': '16',
  'S2': '47',
  'S3': '22',
  'S4': '5',
  'S5': '38',
  'S6': '7',
  'S7': '43',
  'S8': '88',
  'corr': '89',
  'incorr': '90',
  'text': '16 47 22 5 38 7 43 88'},
 {'S1': '59',
  'S2': '50',
  'S3': '28',
  'S4': '5',
  'S5': '39',
  'S6': '61',
  'S7': '91',
  'S8': '52',
  'corr': '88',
  'incorr': '89',
  'text': '59 50 28 5 39 61 91 52'},
 {'S1': '93',
  'S2': '85',
  'S3': '72',
  'S4': '63',
  'S5': '21',
  'S6': '86',
  'S7': '4

# Ablation Expm Functions

In [51]:
from torch import Tensor

def logits_to_ave_logit_diff_2(logits: Float[Tensor, "batch seq d_vocab"], dataset: Dataset, per_prompt=False):
    '''
    Returns logit difference between the correct and incorrect answer.

    If per_prompt=True, return the array of differences rather than the average.
    '''

    # Only the final logits are relevant for the answer
    # Get the logits corresponding to the indirect object / subject tokens respectively
    io_logits: Float[Tensor, "batch"] = logits[range(logits.size(0)), dataset.word_idx["end"], dataset.io_tokenIDs]
    s_logits: Float[Tensor, "batch"] = logits[range(logits.size(0)), dataset.word_idx["end"], dataset.s_tokenIDs]
    # Find logit difference
    answer_logit_diff = io_logits - s_logits
    return answer_logit_diff if per_prompt else answer_logit_diff.mean()

In [75]:
model.reset_hooks(including_permanent=True)  #must do this after running with mean ablation hook
ioi_logits_original, ioi_cache = model.run_with_cache(dataset.toks)
orig_score = logits_to_ave_logit_diff_2(ioi_logits_original, dataset)

In [62]:
def mean_ablate_by_lst(lst, model, orig_score, print_output=True):
    CIRCUIT = {
        "number mover": lst,
        # "number mover 10": lst,
        # "number mover 9": lst,
        "number mover 8": lst,
        "number mover 7": lst,
        "number mover 6": lst,
        "number mover 5": lst,
        "number mover 4": lst,
        "number mover 3": lst,
        "number mover 2": lst,
        "number mover 1": lst,
    }

    SEQ_POS_TO_KEEP = {
        "number mover": "end",
        # "number mover 10": "S10",
        # "number mover 9": "S9",
        "number mover 8": "S8",
        "number mover 7": "S7",
        "number mover 6": "S6",
        "number mover 5": "S5",
        "number mover 4": "S4",
        "number mover 3": "S3",
        "number mover 2": "S2",
        "number mover 1": "S1",
    }

    model.reset_hooks(including_permanent=True)  #must do this after running with mean ablation hook

    # ioi_logits_original, ioi_cache = model.run_with_cache(dataset.toks)

    model = ioi_circuit_extraction.add_mean_ablation_hook(model, means_dataset=dataset_2, circuit=CIRCUIT, seq_pos_to_keep=SEQ_POS_TO_KEEP)
    ioi_logits_minimal = model(dataset.toks)

    # orig_score = logits_to_ave_logit_diff_2(ioi_logits_original, dataset)
    new_score = logits_to_ave_logit_diff_2(ioi_logits_minimal, dataset)
    if print_output:
        # print(f"Average logit difference (IOI dataset, using entire model): {orig_score:.4f}")
        # print(f"Average logit difference (IOI dataset, only using circuit): {new_score:.4f}")
        print(f"Average logit difference (circuit / full) %: {100 * new_score / orig_score:.4f}")
    # return new_score
    return 100 * new_score / orig_score

In [76]:
curr_circuit = [(layer, head) for layer in range(12) for head in range(12)]
mean_ablate_by_lst(curr_circuit, model, orig_score, print_output=False).item()

99.99999237060547

In [64]:
def find_circuit_forw(curr_circuit=None, threshold=10):
    # threshold is T, a %. if performance is less than T%, allow its removal
    if curr_circuit == []:
        # Start with full circuit
        curr_circuit = [(layer, head) for layer in range(12) for head in range(12)]

    for layer in range(0, 12):
        for head in range(12):
            if (layer, head) not in curr_circuit:
                continue

            # Copying the curr_circuit so we can iterate over one and modify the other
            copy_circuit = curr_circuit.copy()

            # Temporarily removing the current tuple from the copied circuit
            copy_circuit.remove((layer, head))

            new_score = mean_ablate_by_lst(copy_circuit, model, orig_score, print_output=False).item()

            # print((layer,head), new_score)
            # If the result is less than the threshold, remove the tuple from the original list
            if (100 - new_score) < threshold:
                curr_circuit.remove((layer, head))

                print("\nRemoved:", (layer, head))
                print(new_score)

    return curr_circuit, new_score

In [65]:
def find_circuit_backw(curr_circuit=None, threshold=10):
    # threshold is T, a %. if performance is less than T%, allow its removal
    if curr_circuit == []:
        # Start with full circuit
        curr_circuit = [(layer, head) for layer in range(12) for head in range(12)]

    for layer in range(11, -1, -1):  # go thru all heads in a layer first
        for head in range(12):
            if (layer, head) not in curr_circuit:
                continue

            # Copying the curr_circuit so we can iterate over one and modify the other
            copy_circuit = curr_circuit.copy()

            # Temporarily removing the current tuple from the copied circuit
            copy_circuit.remove((layer, head))

            new_score = mean_ablate_by_lst(copy_circuit, model, orig_score, print_output=False).item()

            # If the result is less than the threshold, remove the tuple from the original list
            if (100 - new_score) < threshold:
                curr_circuit.remove((layer, head))

                print("\nRemoved:", (layer, head))
                print(new_score)

    return curr_circuit, new_score

# Ablate the model and compare with original

### iter fwd backw, threshold 20

In [66]:
threshold = 20
curr_circuit = []
prev_score = 100
new_score = 0
iter = 1
while prev_score != new_score:
    print('\nfwd prune, iter ', str(iter))
    # track changes in circuit as for some reason it doesn't work with scores
    old_circuit = curr_circuit.copy() # save old before finding new one
    curr_circuit, new_score = find_circuit_forw(curr_circuit=curr_circuit, threshold=threshold)
    if curr_circuit == old_circuit:
        break
    print('\nbackw prune, iter ', str(iter))
    # prev_score = new_score # save old score before finding new one
    old_circuit = curr_circuit.copy() # save old before finding new one
    curr_circuit, new_score = find_circuit_backw(curr_circuit=curr_circuit, threshold=threshold)
    if curr_circuit == old_circuit:
        break
    iter += 1


fwd prune, iter  1

Removed: (0, 0)
96.38060760498047

Removed: (0, 1)
82.5677719116211

Removed: (0, 2)
82.70421600341797

Removed: (0, 3)
81.77704620361328

Removed: (0, 4)
82.78357696533203

Removed: (0, 6)
85.57305145263672

Removed: (0, 7)
85.28573608398438

Removed: (0, 8)
81.6546630859375

Removed: (0, 11)
81.86619567871094

Removed: (1, 1)
80.1313705444336

Removed: (1, 2)
80.03694152832031

Removed: (1, 6)
80.6706314086914

Removed: (1, 7)
80.28878784179688

Removed: (1, 8)
82.18318939208984

Removed: (1, 9)
81.25663757324219

Removed: (1, 10)
83.76980590820312

Removed: (2, 3)
81.14581298828125

Removed: (2, 4)
81.83724212646484

Removed: (2, 11)
81.88544464111328

Removed: (3, 0)
83.1378173828125

Removed: (3, 1)
80.99207305908203

Removed: (3, 4)
81.99833679199219

Removed: (3, 8)
82.0475082397461

Removed: (3, 9)
80.53414916992188

Removed: (3, 10)
81.74751281738281

Removed: (4, 0)
81.10719299316406

Removed: (4, 3)
81.47115325927734

Removed: (4, 5)
81.57735443115234

R

In [67]:
curr_circuit

[(0, 5),
 (1, 0),
 (1, 5),
 (2, 0),
 (2, 1),
 (2, 2),
 (2, 5),
 (2, 6),
 (2, 7),
 (2, 8),
 (2, 9),
 (3, 3),
 (3, 5),
 (3, 6),
 (3, 7),
 (3, 11),
 (4, 1),
 (4, 2),
 (4, 6),
 (4, 8),
 (4, 10),
 (5, 0),
 (5, 1),
 (5, 6),
 (5, 8),
 (5, 9),
 (6, 8),
 (6, 10),
 (7, 6),
 (7, 8),
 (7, 9),
 (7, 10),
 (7, 11),
 (8, 0),
 (8, 1),
 (8, 5),
 (8, 6),
 (8, 7),
 (8, 8),
 (9, 1),
 (9, 3),
 (9, 5),
 (9, 11),
 (10, 7),
 (11, 0),
 (11, 1),
 (11, 7),
 (11, 8),
 (11, 9)]

In [68]:
fb_20 = curr_circuit.copy()
len(fb_20)

49

#### loop rmv and check for most impt heads

In [69]:
circ = fb_20
mean_ablate_by_lst(circ, model, orig_score, print_output=True).item()

Average logit difference (circuit / full) %: 80.3439


80.34394836425781

In [70]:
lh_scores = {}
for lh in circ:
    copy_circuit = circ.copy()
    copy_circuit.remove(lh)
    print("removed: " + str(lh))
    new_score = mean_ablate_by_lst(copy_circuit, model, orig_score, print_output=True).item()
    lh_scores[lh] = new_score

removed: (0, 5)
Average logit difference (circuit / full) %: 75.4070
removed: (1, 0)
Average logit difference (circuit / full) %: 60.9708
removed: (1, 5)
Average logit difference (circuit / full) %: 58.1626
removed: (2, 0)
Average logit difference (circuit / full) %: 78.0082
removed: (2, 1)
Average logit difference (circuit / full) %: 76.2738
removed: (2, 2)
Average logit difference (circuit / full) %: 44.5944
removed: (2, 5)
Average logit difference (circuit / full) %: 78.3988
removed: (2, 6)
Average logit difference (circuit / full) %: 73.2227
removed: (2, 7)
Average logit difference (circuit / full) %: 76.7953
removed: (2, 8)
Average logit difference (circuit / full) %: 73.8933
removed: (2, 9)
Average logit difference (circuit / full) %: 67.1837
removed: (3, 3)
Average logit difference (circuit / full) %: 51.4896
removed: (3, 5)
Average logit difference (circuit / full) %: 75.9370
removed: (3, 6)
Average logit difference (circuit / full) %: 68.8061
removed: (3, 7)
Average logit diff

In [71]:
dict(sorted(lh_scores.items(), key=lambda item: item[1]))

{(4, 10): -7.344493865966797,
 (7, 11): 10.858027458190918,
 (7, 10): 43.18928527832031,
 (6, 10): 44.217735290527344,
 (2, 2): 44.59440994262695,
 (9, 5): 45.4736213684082,
 (5, 8): 46.33286666870117,
 (5, 1): 46.9207649230957,
 (3, 3): 51.489646911621094,
 (7, 6): 52.928993225097656,
 (3, 7): 53.48268508911133,
 (1, 5): 58.162567138671875,
 (8, 0): 58.78353500366211,
 (4, 6): 59.800315856933594,
 (6, 8): 60.119991302490234,
 (1, 0): 60.970829010009766,
 (5, 9): 61.743324279785156,
 (9, 3): 66.15119171142578,
 (2, 9): 67.18372344970703,
 (3, 6): 68.80607604980469,
 (8, 6): 69.43566131591797,
 (8, 5): 70.15142822265625,
 (11, 0): 70.94245910644531,
 (9, 1): 71.47088623046875,
 (11, 8): 72.57658386230469,
 (5, 0): 73.01366424560547,
 (2, 6): 73.22274780273438,
 (2, 8): 73.89334106445312,
 (5, 6): 74.00653839111328,
 (4, 8): 74.91043853759766,
 (8, 8): 74.96321868896484,
 (0, 5): 75.4069595336914,
 (3, 5): 75.93695068359375,
 (10, 7): 76.26075744628906,
 (2, 1): 76.27378845214844,
 (8, 7

In [73]:
lh = (4, 10)
copy_circuit = circ.copy()
copy_circuit.remove(lh)

lst = copy_circuit
CIRCUIT = {
        "number mover": lst,
        # "number mover 10": lst,
        # "number mover 9": lst,
        "number mover 8": lst,
        "number mover 7": lst,
        "number mover 6": lst,
        "number mover 5": lst,
        "number mover 4": lst,
        "number mover 3": lst,
        "number mover 2": lst,
        "number mover 1": lst,
    }

SEQ_POS_TO_KEEP = {
    "number mover": "end",
    # "number mover 10": "S10",
    # "number mover 9": "S9",
    "number mover 8": "S8",
    "number mover 7": "S7",
    "number mover 6": "S6",
    "number mover 5": "S5",
    "number mover 4": "S4",
    "number mover 3": "S3",
    "number mover 2": "S2",
    "number mover 1": "S1",
}

model.reset_hooks(including_permanent=True)  #must do this after running with mean ablation hook
new_score = mean_ablate_by_lst(copy_circuit, model, orig_score, print_output=True).item()
model = ioi_circuit_extraction.add_mean_ablation_hook(model, means_dataset=dataset_2, circuit=CIRCUIT, seq_pos_to_keep=SEQ_POS_TO_KEEP)
ioi_logits_minimal = model(dataset.toks)
new_score = logits_to_ave_logit_diff_2(ioi_logits_minimal, dataset)
new_score

Average logit difference (circuit / full) %: -7.3445


tensor(-0.1915, device='cuda:0')

In [74]:
orig_score

tensor(2.6070, device='cuda:0')